In [ ]:
! pip install -U pip
! pip install -U clearml==0.16.2rc0

In [ ]:
from clearml import Task, OutputModel

In [ ]:
task = Task.init(project_name='Tabular Example', task_name='pick best model')
configuration_dict = {'train_tasks_ids': ['c9bff3d15309487a9e5aaa00358ff091', 'c9bff3d15309487a9e5aaa00358ff091']}
configuration_dict = task.connect(configuration_dict)  # enabling configuration override by clearml
print(configuration_dict)  # printing actual configuration (after override in remote mode)

In [ ]:
results = {}
for task_id in configuration_dict.get('train_tasks_ids'):
    train_task = Task.get_task(task_id)
    results[task_id] = train_task.get_last_scalar_metrics()['accuracy']['total']['last']

In [ ]:
print(results)

In [ ]:
best_model_task_id = max(results.items(), key=lambda x: x[1])[0]
best_model_id = Task.get_task(best_model_task_id).output_model_id

In [ ]:
OutputModel(base_model_id=best_model_id)